# **Notebook de Preparação de Dados para Projeto de Identificação de Depressão**

**Introdução:**
Neste notebook, iremos preparar os dados para a construção de modelos de machine learning. Os atributos utlizados foram selecionados criteriozamente utilizando como base o método CAPTO para selecionar os atributos mais relevantes para a classificação de pessoas que possam possuir depressão. Os dados utilizados serão provenientes da Pesquisa Nacional de Saúde (PNS) de 2019.

**Objetivos:**
O principal objetivo deste notebook é realizar a preparação inicial dos dados, incluindo o tratamento de valores ausentes e vazios, conversão de tipos de variáveis, junção de atributos e discretizarção dos dados, além da análise exploratória dos dados filtrados.


## Bibliotecas Utilizadas
As principais bibliotecas utilizadas nesta etapa são:
- Pandas Profiling: Faz uma análise exploratória automática dos dados, gerando insights sobre problemas nos dados.
- NumPy: Biblioteca fundamental para computação científica em Python.
- Pandas: Biblioteca popular para análise de dados.
- Seaborn / Matplotlib: Visualização de dados.
- IPython Widgets: Permite interatividade.

In [28]:
import numpy as np
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt 

from ydata_profiling import ProfileReport
from ipywidgets import interact, widgets 

from sklearn.preprocessing import scale, minmax_scale, power_transform

In [15]:
sns.set_theme(
    context = 'talk', 
    style='ticks',
    font_scale= 8,
    rc= {
        'figure.figsize': (12,8) 
    }
)

# Análise Inicial da Base de dados

### Importando os dados 
> Os dados filtrados estão em formato .csv 

In [16]:
df = pd.read_csv('linhas_e_colunas_filtradas.csv')


> Visualização inicial dos dados 

In [17]:
df.head()

,C006,C00702,C00703,C008,M01401,M01501,E01602,E01601,E01603,E01604,...,P01601,P018,P019,P02002,P02102,P023,P02401,P02501,P02602,P02601
0,2.0,11.0,1963.0,55.0,3.0,0.0,100.0,1.0,NaN,NaN,...,0.0,3.0,NaN,1.0,2.0,7.0,2.0,0.0,0.0,3.0
1,2.0,11.0,1973.0,45.0,1.0,1.0,1000.0,1.0,NaN,NaN,...,7.0,7.0,2.0,0.0,NaN,0.0,NaN,0.0,0.0,4.0
2,2.0,8.0,1960.0,58.0,1.0,0.0,240.0,1.0,NaN,NaN,...,3.0,3.0,NaN,3.0,2.0,0.0,NaN,0.0,0.0,3.0
3,2.0,99.0,9999.0,41.0,3.0,0.0,998.0,1.0,NaN,NaN,...,7.0,7.0,2.0,1.0,2.0,7.0,1.0,1.0,0.0,3.0
4,2.0,12.0,1966.0,52.0,2.0,1.0,250.0,1.0,NaN,NaN,...,3.0,3.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,3.0


> Vizualizando quantas colunas e linhas temos 

In [18]:
# Obtendo o número de linhas e colunas 
num_linhas, num_colunas = df.shape

print(f'Possuimos {num_linhas} linhas e {num_colunas} colunas.')


Possuimos 40359 linhas e 138 colunas.


> - Removendo a coluna `C008`(Idade): 
> Utilizamos a idade somente para filtrar nossos dados no escopo escolhido, a mesma **não** será relevate para nosso modelo de machine learning. 

In [27]:
df = df.drop(columns=['C008'])

> Verificando quantidade de valores nulos 

In [29]:
# Filtrar colunas com valores nulos maiores que zero
colunas_com_nulos = df.columns[df.isna().sum() > 0]

# Iterar sobre essas colunas e imprimir as informações apenas se o número de nulos for maior que zero
for coluna in colunas_com_nulos:
    num_nulos = df[coluna].isna().sum()
    print(f'Coluna "{coluna}" tem {num_nulos} valores nulos.')



Coluna "E01602" tem 14459 valores nulos.
Coluna "E01601" tem 14459 valores nulos.
Coluna "E01603" tem 40174 valores nulos.
Coluna "E01604" tem 40174 valores nulos.
Coluna "E01605" tem 40355 valores nulos.
Coluna "E01801" tem 38923 valores nulos.
Coluna "E01802" tem 38923 valores nulos.
Coluna "E01803" tem 40334 valores nulos.
Coluna "E01804" tem 40334 valores nulos.
Coluna "E01805" tem 40351 valores nulos.
Coluna "F001021" tem 31524 valores nulos.
Coluna "F007021" tem 39261 valores nulos.
Coluna "F008021" tem 38334 valores nulos.
Coluna "VDF00102" tem 35087 valores nulos.
Coluna "E01201" tem 14074 valores nulos.
Coluna "E01401" tem 14074 valores nulos.
Coluna "E014011" tem 40064 valores nulos.
Coluna "E002" tem 23903 valores nulos.
Coluna "E019" tem 38893 valores nulos.
Coluna "E017" tem 14074 valores nulos.
Coluna "M005011" tem 37214 valores nulos.
Coluna "M00601" tem 37214 valores nulos.
Coluna "D00901" tem 3763 valores nulos.
Coluna "Q09201" tem 35765 valores nulos.
Coluna "Q09202" 

> Conferindo tipos de dados 

In [30]:
df.dtypes

C006      category
C00702     float64
C00703     float64
M01401     float64
M01501     float64
            ...   
P023       float64
P02401     float64
P02501     float64
P02602     float64
P02601    category
Length: 137, dtype: object

>Observamos que todas as variáveis estão sendo considerados **float**, oque não condiz com a realidade. 

> Vamos converte-las para variáveis do tipo **categorico** e manter somente as variaveis que realmente são numéricas como **float**

In [21]:
df[ [
  'C008','C00702', 'C00703',
  'W00101', 'W00102', 'W00201', 'W00202',  # Peso, altura e idade
  'A01001','A011',
  'M01401', 'M01501', 'M005011','P029','P02801',
  'E01602', 'E01601', 'E01605', 'E01802', 'E01804', 'E019', 'E017',
  'F001021', 'F007021', 'F008021', 'VDF00102','P03702','P03701', 'P006', 
  'P00901', 'P01001', 'P01101', 'P013','P015', 'P02001', 'P02101',  'P01601','P018', 'P019',  'P02602',
  'P02002', 'P02102',  'P023', 'P02401', 'P02501', 'P02602', 'P00403', 'P00404', 'P00103', 'P00104'
]]

,C008,C00702,C00703,W00101,W00102,W00201,W00202,A01001,A011,M01401,...,P02002,P02102,P023,P02401,P02501,P02602,P00403,P00404,P00103,P00104
0,55.0,11.0,1963.0,NaN,NaN,NaN,NaN,5.0,2.0,3.0,...,1.0,2.0,7.0,2.0,0.0,0.0,145.0,145.0,60.0,60.0
1,45.0,11.0,1973.0,NaN,NaN,NaN,NaN,8.0,4.0,1.0,...,0.0,NaN,0.0,NaN,0.0,0.0,155.0,155.0,77.0,77.0
2,58.0,8.0,1960.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,3.0,2.0,0.0,NaN,0.0,0.0,NaN,159.0,NaN,56.0
3,41.0,99.0,9999.0,NaN,NaN,NaN,NaN,6.0,2.0,3.0,...,1.0,2.0,7.0,1.0,1.0,0.0,170.0,170.0,75.0,75.0
4,52.0,12.0,1966.0,NaN,NaN,NaN,NaN,2.0,1.0,2.0,...,0.0,NaN,0.0,NaN,0.0,0.0,149.0,149.0,52.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40354,51.0,9.0,1967.0,NaN,NaN,NaN,NaN,5.0,1.0,2.0,...,0.0,NaN,2.0,2.0,0.0,0.0,158.0,158.0,62.0,62.0
40355,44.0,5.0,1975.0,NaN,NaN,NaN,NaN,5.0,3.0,3.0,...,1.0,2.0,1.0,2.0,7.0,0.0,162.0,162.0,62.0,62.0
40356,54.0,10.0,1964.0,NaN,NaN,NaN,NaN,5.0,2.0,3.0,...,1.0,2.0,0.0,NaN,0.0,0.0,168.0,168.0,63.0,63.0
40357,44.0,7.0,1975.0,NaN,NaN,NaN,NaN,5.0,2.0,2.0,...,0.0,NaN,0.0,NaN,0.0,0.0,168.0,168.0,72.0,72.0


In [23]:
# Converter todas as colunas para categóricas
df = df.astype('category')

# Lista das colunas que devem ser numéricas
colunas_numericas = [
    'C008', 'C00702', 'C00703',
    'W00101', 'W00102', 'W00201', 'W00202',  # Peso, altura e idade
    'A01001', 'A011',
    'M01401', 'M01501', 'M005011', 'P029', 'P02801',
    'E01602', 'E01601', 'E01605', 'E01802', 'E01804', 'E019', 'E017',
    'F001021', 'F007021', 'F008021', 'VDF00102', 'P03702', 'P03701', 'P006',
    'P00901', 'P01001', 'P01101', 'P013', 'P015', 'P02001', 'P02101', 'P01601', 'P018', 'P019', 'P02602',
    'P02002', 'P02102', 'P023', 'P02401', 'P02501', 'P02602', 'P00403', 'P00404', 'P00103', 'P00104'
]

# Tentar converter as colunas selecionadas para numérico
for coluna in colunas_numericas:
    try:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
    except ValueError:
        print(f"A coluna {coluna} contém valores não numéricos e não pode ser convertida.")

# Verificar os tipos de dados após a conversão
print(df.dtypes)


C006      category
C00702     float64
C00703     float64
C008       float64
M01401     float64
            ...   
P023       float64
P02401     float64
P02501     float64
P02602     float64
P02601    category
Length: 138, dtype: object


> Agora que nossos tipos de dados estão corretos, iremos gerar um relatório para análise e melhor compreenção de nossos dados. 

In [11]:
# Gerando Relatorio para Análises Gerais
#profile = ProfileReport(df, title="Profiling Report")
#profile.to_file(r"C:\Users\maype\OneDrive\Área de Trabalho\projects\projeto-mineracao-de-dados\Vizualizaçoes\EDA_pns2019_depressao.html")

# Tratando dados 


## Dados duplicados
> Verificando se temos valores duplicados 

In [24]:
# Identificando Valores Duplicados
df[df.duplicated(keep=False)]

,C006,C00702,C00703,C008,M01401,M01501,E01602,E01601,E01603,E01604,...,P01601,P018,P019,P02002,P02102,P023,P02401,P02501,P02602,P02601
25251,1.0,1.0,1960.0,59.0,3.0,2.0,12000.0,1.0,NaN,NaN,...,0.0,0.0,NaN,2.0,2.0,7.0,1.0,0.0,0.0,3.0
25253,1.0,1.0,1960.0,59.0,3.0,2.0,12000.0,1.0,NaN,NaN,...,0.0,0.0,NaN,2.0,2.0,7.0,1.0,0.0,0.0,3.0


> Agora que idetificamos duplicatas em nossos dados, vamos remove-las 


In [25]:
#Removendo valores duplicados 
df.drop_duplicates()

,C006,C00702,C00703,C008,M01401,M01501,E01602,E01601,E01603,E01604,...,P01601,P018,P019,P02002,P02102,P023,P02401,P02501,P02602,P02601
0,2.0,11.0,1963.0,55.0,3.0,0.0,100.0,1.0,NaN,NaN,...,0.0,3.0,NaN,1.0,2.0,7.0,2.0,0.0,0.0,3.0
1,2.0,11.0,1973.0,45.0,1.0,1.0,1000.0,1.0,NaN,NaN,...,7.0,7.0,2.0,0.0,NaN,0.0,NaN,0.0,0.0,4.0
2,2.0,8.0,1960.0,58.0,1.0,0.0,240.0,1.0,NaN,NaN,...,3.0,3.0,NaN,3.0,2.0,0.0,NaN,0.0,0.0,3.0
3,2.0,99.0,9999.0,41.0,3.0,0.0,998.0,1.0,NaN,NaN,...,7.0,7.0,2.0,1.0,2.0,7.0,1.0,1.0,0.0,3.0
4,2.0,12.0,1966.0,52.0,2.0,1.0,250.0,1.0,NaN,NaN,...,3.0,3.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40354,2.0,9.0,1967.0,51.0,2.0,0.0,1200.0,1.0,NaN,NaN,...,7.0,7.0,3.0,0.0,NaN,2.0,2.0,0.0,0.0,3.0
40355,1.0,5.0,1975.0,44.0,3.0,0.0,1200.0,1.0,NaN,NaN,...,0.0,2.0,NaN,1.0,2.0,1.0,2.0,7.0,0.0,3.0
40356,2.0,10.0,1964.0,54.0,3.0,3.0,4000.0,1.0,NaN,NaN,...,7.0,2.0,NaN,1.0,2.0,0.0,NaN,0.0,0.0,4.0
40357,1.0,7.0,1975.0,44.0,2.0,2.0,2000.0,1.0,NaN,NaN,...,2.0,3.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,4.0


In [87]:
# Exportando a base de dados 
df.to_csv("C:base_tratada.csv", index= False)